# Проект: создание собственного трансформера DRCTransformer

**Описание проекта:**

Данный проект направлен на создание собственного трансформера `DRCTransformer`, для создания более робастного пайплайна, чтобы модель машинного обучения могла работать, как с выбросами в данных, так и с `OOD`.

>**DRCTransformer** — Dynamic Range Compression for Robust Feature Scaling. A sklearn-compatible transformer that gently compresses outliers and out-of-distribution (OOD) samples while preserving the underlying data structure, inspired by audio dynamic processing.
>
>**DRCTransformer** — Сжатие динамического диапазона для надежного масштабирования функций. Совместимый со sklearn преобразователь, который мягко сжимает выбросы и выборки, не относящиеся к распределению (OOD), сохраняя при этом базовую структуру данных, основанную на динамической обработке звука.

## Импорт библиотек

In [5]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

## Разработка

### Логика и алгоритм

**Опиши математически или алгоритмически, как именно работает твоя компрессия:**

- Как ты определяешь выбросы?
- Как происходит "сжатие" — линейное, нелинейное преобразование?
- Сохраняется ли порядок значений? Среднее/медиана/дисперсия?
- Применим ли метод к одному признаку или к многомерным данным?
- Убедись, что метод обратим или, если нет — чётко опиши, что это «lossy» преобразование.

**Ответы на поставленные вопросы:**

- Выбросы определяются самим разработчиком, `OOD` определяются методом `fit()`, как минимальное/максимальное/минимальное и максимальное значение.
- Сжатие происходит нелинейно, с помощью степенного преобразования, либо логарифмического, в зависимости от режима.
- Среднее и медиана сохраняются, дисперсия ождиаемо уменьшится, чаще всего под влияние попадают значения выше `95`- процентиля.
- Данный метод применим, как к одному признаку, так и к многомерным данным (к каждому признаку индивидуально).
- Метод может быть обратим.

#### Базовый алгоритм

In [2]:
# Функция для компрессии данных
def compress(df, threshold, coef, method='power', dry=0):
    '''
    Функция для степенной копрессии данных,
    пики превышающие порог масштабируются c
    соответствнно размеру коэффициента.
    
    Аргументы:
    df - pd.Series (оригинальный признак для компрессии);
    threshold - порог срабатывания компрессора (передавать абсолютные значение);
    coef - степень сжатия (для метода power), 
    коэффициент ослабления силы сжатия (для метода log);
    method - метод сжатия (степенной - power, логарифмический - log);
    dry - доля подмешивания оригинальных значений, для ослабления сжатия.

    Вывод: pd.Series.
    '''
    mask = df > threshold
    compress_col = df.copy().astype('float64')
    
    if method == 'power':
        
        compress_col[mask] = threshold + (df[mask] - threshold) ** coef
    
    elif method == 'log':
        
        compress_col[mask] = threshold + np.log1p(df[mask] - threshold) * coef
    
    else:
        raise ValueError(f"Неподдерживаемый метод: '{method}'. Допустимые значения: 'power', 'log'")
    
    return df * dry + compress_col * (1 - dry)

### Разработка трансформера

In [4]:
# Код трансформера
class DRCTransformer(BaseEstimator, TransformerMixin):

    # Инициализация трансформера
    def __init__(self, coef=0.5, threshold=None, dry=0.0, method='power'):
        '''
        Параметры:
        - coef: степень сжатия (float) или словарь {col: coef}
        - threshold: порог (float или dict). Если None — будет вычислен как максимум колонки при fit.
        - dry: доля оригинального сигнала (0.0 = полное сжатие, 1.0 = без изменений)
        - method: 'power' или 'log'
        '''
        self.coef = coef
        self.threshold = threshold
        self.dry = dry
        self.method = method

    # Функция компрессии
    def _compress_array(self, data, threshold, coef, dry, method):
        '''
        Применяет компрессию к одному признаку (1D numpy array).
        Возвращает сжатый массив той же длины.
        '''
        data = np.asarray(data, dtype=np.float64)
        mask = data > threshold
        compressed = data.copy()
    
        if method == 'power':
            compressed[mask] = threshold + np.power(data[mask] - threshold, coef)
    
        else:
            compressed[mask] = threshold + np.log1p(data[mask] - threshold) * coef
    
        return dry * data + (1 - dry) * compressed

    # Вспомогательная функция _expand_param
    def _expand_param(self, param, n_features, default):
        '''
        Преобразование заданных параметров.
        '''
        if param is None:
            '''
            Если параметр не задан он определяется, 
            как значение по умолчанию
            '''
            return [default] * n_features
        elif isinstance(param, dict):
            '''
            Берем значения из словаря, если значение отсутствует,
            берем значение по умолчанию
            '''
            return [param.get(name, default) for name in self.feature_names_in_]
        elif np.isscalar(param):
            '''
            Если параметр задан одним числом, распределяем 
            это значение на все признаки
            '''
            return [param] * n_features
        else:
            '''
            Если значения параметра заданы массивом или списком,
            распределяем значения по порядку
            '''
            param = list(param)
            if len(param) != n_features:
                raise ValueError(f'Длина параметра не совпадает с числом признаков ({n_features}).')
            return param

    # Метод get_feature_names_out()
    def get_feature_names_out(self, input_features=None):
        '''
        Возвращение имен входных признаков
        '''
        check_is_fitted(self, 'feature_names_in_')
        return np.array(self.feature_names_in_, dtype=object)
            
    # Метод fit()
    def fit(self, X, y=None):
        '''
        Обучение трансформера.
        Определяем количество колонок и значения параметров для них
        '''
        # Сохраняем информацию о колонках, если X — DataFrame
        if isinstance(X, pd.DataFrame):
            self.feature_names_in_ = X.columns.to_list()
            X_values = X.values
        else:
            X_values = np.asarray(X)
            # Если вход — массив, даём имена "заглушки"
            self.feature_names_in_ = [f'col_{i}' for i in range(X_values.shape[1])]

        n_features = X_values.shape[1]

        # Обработка threshold
        if self.threshold is None:
            '''
            Если порог не задан он определяется, 
            как максимальное значение признака
            '''
            self.threshold_ = np.max(X_values, axis=0).tolist()
        elif isinstance(self.threshold, dict):
            '''
            Берем значения из словаря, если значения отсутствует,
            берем максимальное значение признака
            '''
            self.threshold_ = [
                self.threshold.get(name, np.max(X_values[:, i]))
                for i, name in enumerate(self.feature_names_in_)
            ]
        elif np.isscalar(self.threshold):
            '''
            Если порог задан одним числом, распределяем 
            это значение на все признаки
            '''
            self.threshold_ = [self.threshold] * n_features
        else:
            '''
            Если значения заданы массивом или списком,
            распределяем значения по порядку
            '''
            self.threshold_ = list(self.threshold)
            if len(self.threshold_) != n_features:
                raise ValueError(f'Длина параметра не совпадает с числом признаков ({n_features}).')

        # Обработка остальных параметров (coef, dry, method)
        self.coef_ = self._expand_param(self.coef, n_features, default=0.5)

        # Валидация параметре coef, должно быть coef > 0
        for i, c in enumerate(self.coef_):
            if c <= 0:
                col_name = self.feature_names_in_[i]
                raise ValueError(f"Параметр coef для колонки '{col_name}' = {c}, должен быть coef > 0.")
        
        self.dry_ = self._expand_param(self.dry, n_features, default=0.0)
        
        # Валидация параметра dry, должно быть [0, 1]
        for i, d in enumerate(self.dry_):
            if not (0 <= d <= 1):
                col_name = self.feature_names_in_[i]
                raise ValueError(f"Параметр dry для колонки '{col_name}' = {d}, должен быть в диапазоне [0, 1].")
                
        self.method_ = self._expand_param(self.method, n_features, default='power')

        # Валидация параметра method, должен быть power или log
        for i, m in enumerate(self.method_):
            if m not in ['power', 'log']:
                col_name = self.feature_names_in_[i]
                raise ValueError(f"Неподдерживаемый метод: '{m}'. Допустимые значения: 'power', 'log'")

        return self     

    # Метод transform()
    def transform(self, X):
        '''
        Применение компрессии к данным
        '''
        # Проверка, обучен ли трансформер
        check_is_fitted(self)
        
        X_values = np.asarray(X)
        X_compressed = X_values.copy()

        for i in range(X_values.shape[1]):
            X_compressed[:, i] = self._compress_array(
                data=X_values[:, i],
                threshold=self.threshold_[i],
                coef=self.coef_[i],
                dry=self.dry_[i],
                method=self.method_[i]
            )
        
        return X_compressed

## Тестирование

In [21]:
X = pd.DataFrame({'A': [1, 2, 10, 15], 'B': [0.1, 0.2, 0.9, 0.95]})

In [22]:
compressor_array = DRCTransformer(threshold={'A': 5}, coef=0.5, dry=0.1)
compressor_pd = DRCTransformer(threshold={'A': 5}, coef=0.5, dry=0.1).set_output(transform='pandas')

In [23]:
# Проверяем, что работает с numpy
print("Transform output type:", type(compressor_array.fit_transform(X)))

# Проверяем set_output
X_trans = compressor_pd.fit_transform(X)
print("With set_output ->", type(X_trans), X_trans.columns.to_list())

Transform output type: <class 'numpy.ndarray'>
With set_output -> <class 'pandas.core.frame.DataFrame'> ['A', 'B']


In [24]:
X_trans

,A,B
0,1.000000,0.10
1,2.000000,0.20
2,7.512461,0.90
3,8.846050,0.95


## Анализ результатов